In [1]:
!pip install pyngrok
!pip install deepface==0.0.75 tensorflow==2.12.0 keras==2.12.0 opencv-python-headless
!pip install -q -U google-genai

In [2]:
import getpass
from pyngrok import conf

print("Enter your ngrok v2 authtoken (from https://dashboard.ngrok.com/auth):")
conf.get_default().auth_token = getpass.getpass()

Enter your ngrok v2 authtoken (from https://dashboard.ngrok.com/auth):
··········


In [3]:
from google import genai

print("Enter your Google GenAI API key:")
client = genai.Client(api_key=getpass.getpass())

Enter your Google GenAI API key:
··········


In [4]:
GEMINI_PROMPT_TEMPLATE = """
You are a friendly AI companion sitting on a user's desktop. Based on the current context, the detected emotion is: "{emotion}".

Your task is to speak directly to the user with a short, warm, and emotionally supportive message.

⚠️ Do not explain what you see in the image or describe facial expressions.
⚠️ Do not mention the image, model, detection, or any analysis.

Just speak like a caring friend or digital buddy who notices the user's mood.

Tone: gentle, kind, and encouraging.
Length: 1–2 sentences. No more than 30 words.
"""


GEMINI_NO_FACE_PROMPT = """
You are a friendly AI companion on the user's desktop. No face was detected in the camera feed.

Still, write a short message to the user — perhaps they’ve stepped away or the camera didn’t catch them.

⚠️ Do not mention the detection or the image.
⚠️ Do not say anything about faces, models, or analysis.

Just offer a light, kind message as if you're keeping them company.

Tone: thoughtful, relaxed, and supportive.
Length: 1–2 sentences. No more than 30 words.
"""



In [5]:
import base64
import google.generativeai as genai

def image_bytes_to_base64_str(img_bytes):
    encoded_bytes = base64.b64encode(img_bytes)
    base64_str = encoded_bytes.decode('utf-8')
    return base64_str


def send_image_to_gemini(image, detected_emotion=None):
    prompt = (
        GEMINI_PROMPT_TEMPLATE.format(emotion=detected_emotion)
        if detected_emotion
        else GEMINI_NO_FACE_PROMPT
    )

    try:

        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[
                image,
                prompt
            ]
        )
        return response.text.strip()
    except Exception as e:
        return f"[Gemini Error] {str(e)}"

In [ ]:

import numpy as np
import cv2
from flask import Flask, request, jsonify
from pyngrok import ngrok
from deepface import DeepFace

from deepface.commons import functions
from IPython.display import display
import matplotlib.pyplot as plt
from PIL import Image
import io

# 1) Choose the port your Flask app will listen on
PORT = 5000

# 2) Open a public ngrok tunnel to that port
public_url = ngrok.connect(PORT, bind_tls=True).public_url
print(f" * ngrok tunnel: {public_url} → http://127.0.0.1:{PORT}")

# 3) Define your Flask app
app = Flask(__name__)

@app.route("/emotion", methods=["POST"])
def emotion():
    img_bytes = request.get_data()
    if not img_bytes:
        return jsonify({
            "status": "error",
            "error": "no image data"
        }), 200

    arr = np.frombuffer(img_bytes, dtype=np.uint8)
    frame = cv2.imdecode(arr, cv2.IMREAD_COLOR)
    if frame is None:
        return jsonify({
            "status": "error",
            "error": "invalid image"
        }), 200

    # Save temp image
    import tempfile, os, traceback
    tmpf = tempfile.NamedTemporaryFile(suffix=".jpg", delete=False)
    cv2.imwrite(tmpf.name, frame)

    try:
        # Preprocess ROI
        roi_img, region = functions.preprocess_face(
            img=tmpf.name,
            detector_backend="opencv",
            enforce_detection=True,
            return_region=True
        )

        roi_img = np.squeeze(roi_img)
        if roi_img.ndim == 2:
            plt.imshow(roi_img, cmap='gray')
        else:
            plt.imshow(roi_img)
        plt.axis('off')
        plt.title("Extracted Face ROI")
        display(plt.gcf())
        plt.close()

        # DeepFace emotion
        res = DeepFace.analyze(
            img_path=tmpf.name,
            actions=["emotion"],
            enforce_detection=True,
            detector_backend="opencv"
        )
        if isinstance(res, list):
            res = res[0]

        dom = res.get("dominant_emotion", "neutral").lower()
        scr = res.get("emotion", {})

        image = Image.open(io.BytesIO(img_bytes))
        gemini_msg = send_image_to_gemini(image, detected_emotion=dom)

        return jsonify({
            "status": "ok",
            "result": {
                "dominant_emotion": dom,
                "gemini_interpretation": gemini_msg
            }
        }), 200

    except ValueError:
        image = Image.open(io.BytesIO(img_bytes))
        gemini_msg = send_image_to_gemini(image, detected_emotion=None)

        return jsonify({
            "status": "reinit_required",
            "error": "Face not detected",
            "gemini_interpretation": gemini_msg
        }), 200

    except Exception as e:
        traceback.print_exc()
        return jsonify({
            "status": "error",
            "error": str(e)
        }), 500

    finally:
        os.remove(tmpf.name)


# 4) Run Flask in foreground (so you see any errors immediately)
app.run(host="0.0.0.0", port=PORT, debug=True, use_reloader=False)



Directory  /root /.deepface created
Directory  /root /.deepface/weights created
 * ngrok tunnel: https://5a53-35-236-242-57.ngrok-free.app → http://127.0.0.1:5000
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Apr/2025 07:05:45] "POST /emotion HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Apr/2025 07:06:38] "POST /emotion HTTP/1.1" 200 -
